In [2]:
import duckdb

from lonboard import Map, PolygonLayer
from lonboard.basemap import CartoBasemap

In [3]:
db = duckdb.connect(database='caba_population_analysis.duckdb', read_only=False)
db.execute("INSTALL spatial; LOAD spatial;")
db.execute("INSTALL httpfs; LOAD httpfs;")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
# db.execute("""INSTALL zipfs from community; LOAD zipfs;""")

#### Census 2010 data

In [5]:
census_radius_2010 = r"https://cdn.buenosaires.gob.ar/datosabiertos/datasets/direccion-general-de-estadisticas-y-censos/informacion-censal-por-radio/caba_radios_censales.geojson"

In [6]:
db.sql(f"""
CREATE OR REPLACE TABLE census_blocks_2010 AS
    SELECT *
    FROM ST_Read('{census_radius_2010}');
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [7]:
db.sql("""
    describe census_blocks_2010;
""")

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ WKT         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ID          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ CO_FRAC_RA  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ COMUNA      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ FRACCION    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ RADIO       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ TOTAL_POB   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ T_VARON     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ T_MUJER     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ T_VIVIENDA  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ V_PARTICUL  │ VARC

#### Census 2001 data

In [8]:
# db.sql("""
# -- Clip source_polygons using clip_polygons
# CREATE TABLE clipped_result AS
# SELECT
#     s.id AS source_id,
#     c.id AS clip_id,
#     ST_Intersection(s.geom, c.geom) AS geom
# FROM
#     source_polygons s
# JOIN
#     clip_polygons c
# ON
#     ST_Intersects(s.geom, c.geom)
# WHERE
#     ST_IsValid(s.geom) AND ST_IsValid(c.geom);
#            """)

https://www.flother.is/til/duckdb-st-read/

In [9]:
census_blocks_2001_shp = r"https://cdn.buenosaires.gob.ar/datosabiertos/datasets/direccion-general-de-estadisticas-y-censos/informacion-censal-por-radio/informacion-censal-por-radio-2001.zip"

In [13]:
db.sql("""
CREATE OR REPLACE TABLE census_blocks_2001 AS
    SELECT *
    --from st_read('/vsicurl/census_blocks_2001_shp/informacion_censal_por_radio_2001.shp', open_options=['GDAL_HTTP_UNSAFESSL=YES'])
    FROM ST_Read('/vsizip/informacion-censal-por-radio-2001.zip/informacion_censal_por_radio_2001.shp')
""")

In [18]:
from pyproj import CRS

In [19]:
blocks_2001_crs = 'PROJCS["Argentina_GKBsAs",GEOGCS["GCS_Campo_Inchauspe",DATUM["D_Campo_Inchauspe",SPHEROID["International_1924",6378388.0,297.0]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",100000.0],PARAMETER["False_Northing",100000.0],PARAMETER["Central_Meridian",-58.4627],PARAMETER["Scale_Factor",0.999998],PARAMETER["Latitude_Of_Origin",-34.6297166],UNIT["Meter",1.0]]'

In [21]:
caba_projcrs = CRS.from_wkt(blocks_2001_crs)

In [17]:
db.sql("""
    summarize census_blocks_2001;
       """)

┌─────────────┬─────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬───────┬─────────────────┐
│ column_name │ column_type │                                                                                                                                                                                                    min      

In [42]:
blocks_2010 = PolygonLayer.from_duckdb(
    sql="SELECT geom FROM census_blocks_2010",
    con=db,
    get_fill_color=[50, 25, 0, 180],
    get_line_color=[255, 0, 0],
    get_line_width=1,
    pickable=True,
    extruded=False,
    auto_highlight=True
)

c:\Users\learo\MyDocs\GitHub\geopython\.venv\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:37: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


In [43]:
blocks_2001 = PolygonLayer.from_duckdb(
    sql=f"SELECT st_transform(geom,'{blocks_2001_crs}', 'EPSG:4326', always_xy:=TRUE) as geom FROM census_blocks_2001",
    con=db,
    get_fill_color=[200, 0, 100, 180],
    get_line_color=[0, 255, 0],
    get_line_width=1,
    pickable=True,
    extruded=False,
    auto_highlight=True
)

In [45]:
m = Map(
    layers=[blocks_2010, blocks_2001],
    basemap_style=CartoBasemap.Voyager
)
m

Map(basemap_style=<CartoBasemap.Voyager: 'https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json'>, cust…